# --- Day 22: Wizard Simulator 20XX ---

Little Henry Case decides that defeating bosses with **swords and stuff** is boring. Now he's playing the game with a wizard. Of course, he gets stuck on another boss and needs your help again.

In this version, combat still proceeds with the player and the boss taking alternating turns. The player still goes first. Now, however, you don't get any equipment; instead, you must choose one of your spells to cast. The first character at or below `0` hit points loses.

Since you're a wizard, you don't get to wear armor, and you can't attack normally. However, since you do **magic damage**, your opponent's armor is ignored, and so the boss effectively has zero armor as well. As before, if armor (from a spell, in this case) would reduce damage below `1`, it becomes `1` instead - that is, the boss' attacks always deal at least `1` damage.

On each of your turns, you must select one of your spells to cast. If you cannot afford to cast any spell, you lose. Spells cost **mana**; you start with **500** mana, but have no maximum limit. You must have enough mana to cast a spell, and its cost is immediately deducted when you cast it. Your spells are Magic Missile, Drain, Shield, Poison, and Recharge.

+ **Magic Missile** costs `53` mana. It instantly does `4` damage.
+ **Drain** costs `73` mana. It instantly does `2` damage and heals you for `2` hit points.
+ **Shield** costs `113` mana. It starts an **effect** that lasts for `6` turns. While it is active, your armor is increased by `7`.
+ **Poison** costs `173` mana. It starts an **effect** that lasts for `6` turns. At the start of each turn while it is active, it deals the boss `3` damage.
+ **Recharge** costs `229` mana. It starts an **effect** that lasts for `5` turns. At the start of each turn while it is active, it gives you `101` new mana.

**Effects** all work the same way. Effects apply at the start of both the player's turns and the boss' turns. Effects are created with a timer (the number of turns they last); at the start of each turn, after they apply any effect they have, their timer is decreased by one. If this decreases the timer to zero, the effect ends. You cannot cast a spell that would start an effect which is already active. However, effects can be started on the same turn they end.

For example, suppose the player has `10` hit points and `250` mana, and that the boss has `13` hit points and `8` damage:

```
-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 13 hit points
Player casts Poison.

-- Boss turn --
- Player has 10 hit points, 0 armor, 77 mana
- Boss has 13 hit points
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 damage.

-- Player turn --
- Player has 2 hit points, 0 armor, 77 mana
- Boss has 10 hit points
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile, dealing 4 damage.

-- Boss turn --
- Player has 2 hit points, 0 armor, 24 mana
- Boss has 3 hit points
Poison deals 3 damage. This kills the boss, and the player wins.
```

Now, suppose the same initial conditions, except that the boss has `14` hit points instead:

```
-- Player turn --
- Player has 10 hit points, 0 armor, 250 mana
- Boss has 14 hit points
Player casts Recharge.

-- Boss turn --
- Player has 10 hit points, 0 armor, 21 mana
- Boss has 14 hit points
Recharge provides 101 mana; its timer is now 4.
Boss attacks for 8 damage!

-- Player turn --
- Player has 2 hit points, 0 armor, 122 mana
- Boss has 14 hit points
Recharge provides 101 mana; its timer is now 3.
Player casts Shield, increasing armor by 7.

-- Boss turn --
- Player has 2 hit points, 7 armor, 110 mana
- Boss has 14 hit points
Shield's timer is now 5.
Recharge provides 101 mana; its timer is now 2.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 1 hit point, 7 armor, 211 mana
- Boss has 14 hit points
Shield's timer is now 4.
Recharge provides 101 mana; its timer is now 1.
Player casts Drain, dealing 2 damage, and healing 2 hit points.

-- Boss turn --
- Player has 3 hit points, 7 armor, 239 mana
- Boss has 12 hit points
Shield's timer is now 3.
Recharge provides 101 mana; its timer is now 0.
Recharge wears off.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 2 hit points, 7 armor, 340 mana
- Boss has 12 hit points
Shield's timer is now 2.
Player casts Poison.

-- Boss turn --
- Player has 2 hit points, 7 armor, 167 mana
- Boss has 12 hit points
Shield's timer is now 1.
Poison deals 3 damage; its timer is now 5.
Boss attacks for 8 - 7 = 1 damage!

-- Player turn --
- Player has 1 hit point, 7 armor, 167 mana
- Boss has 9 hit points
Shield's timer is now 0.
Shield wears off, decreasing armor by 7.
Poison deals 3 damage; its timer is now 4.
Player casts Magic Missile, dealing 4 damage.

-- Boss turn --
- Player has 1 hit point, 0 armor, 114 mana
- Boss has 2 hit points
Poison deals 3 damage. This kills the boss, and the player wins.
```

You start with **50 hit points** and **500 mana points**. The boss's actual stats are in your puzzle input. What is the **least amount of mana** you can spend and still win the fight? (Do not include mana recharge effects as "spending" negative mana.)

In [26]:
# Ref : https://www.reddit.com/r/adventofcode/comments/3xspyl/day_22_solutions/

spell_cost = {'M': 53, 'D': 73, 'S': 113, 'P': 173, 'R': 229}

def sim(actions, part):
    boss = {
        'hp' : 55,
        'dmg': 8
    }

    player = {
        'hp' : 50, 
        'mana': 500, 
        'armor': 0
    }

    turn, turn_c = 0, 00
    mana_spent = 0
    poison_left, shield_left, recharge_left = 0, 0, 0
    my_turn = 1
    
    
    while True:
        if len(actions)-1 < turn_c:
            print('out of moves')
            return 0
        if poison_left:
            poison_left = max(poison_left - 1, 0)
            boss['hp'] -= 3
        if shield_left:
            shield_left = max(shield_left - 1, 0)
            player['armor'] = 7
        else:
            player['armor'] = 0        
        if recharge_left:
            recharge_left = max(recharge_left - 1, 0)
            player['mana'] += 101
        if my_turn == 1:
            if part == 2:
                player['hp'] -= 1
                if player['hp'] <= 0:
                    return 0
            action = actions[turn_c]
            player['mana'] -= spell_cost[action]
            mana_spent += spell_cost[action]
            if action == 'M':
                boss['hp'] -= 4
            elif action == 'D':
                boss['hp'] -= 2
                player['hp'] += 2
            elif action == 'S':
                if shield_left:
                    return 0
                shield_left = 6
            elif action == 'P':
                if poison_left:
                    return 0
                poison_left = 6
            elif action == 'R':
                if recharge_left:
                    return 0
                recharge_left = 5
            if player['mana'] < 0:
                return 0
        if boss['hp'] <= 0:
            return mana_spent
        if my_turn == -1:
            player['hp'] -= max(boss['dmg'] - player['armor'], 1)
            if player['hp'] <= 0:
                return 0
        if my_turn == 1:
            turn_c += 1
        my_turn = -my_turn
        turn += 1

def iterate_actions(pos):
    actions[pos] = 'DSPRM'['MDSPR'.index(actions[pos])]
    if actions[pos] == 'M':
        if pos+1 <= len(actions):
            iterate_actions(pos+1)

for part in (1, 2):
    print(part)
    actions = ['M'] * 20
    min_spent = 1000000
    for i in range(1000000):
        result = sim(actions, part)
        if result:
            min_spent = min(result, min_spent)
        iterate_actions(0)    
    print(min_spent)




1000000
1000000
1
953
2
1289


## --- Part Two ---

On the next run through the game, you increase the difficulty to **hard**.

At the start of each **player turn** (before any other effects apply), you lose `1` hit point. If this brings you to or below `0` hit points, you lose.

With the same starting stats for you and the boss, what is the **least amount of mana** you can spend and still win the fight?